# 시맨틱 검색 엔진 구축하기
이 노트북은 LangChain의 문서 로더, 임베딩, 그리고 벡터 스토어 추상화 개념에 익숙해지도록 설계되었습니다. 이러한 추상화는 (벡터) 데이터베이스 및 기타 소스에서 데이터를 검색하여 LLM 워크플로우에 통합하도록 지원합니다. 이는 **RAG (Retrieval-Augmented Generation)** 와 같은 모델 추론의 일부로 데이터를 검색해 활용하는 애플리케이션에 중요합니다.

이 노트북에서는 PDF 문서에 대한 검색 엔진을 구축합니다. 이를 통해 입력 쿼리와 유사한 PDF의 특정 구절을 검색할 수 있습니다.

### 개념
이 가이드는 텍스트 데이터 검색에 중점을 둡니다. 아래 개념을 다룰 것입니다:

- 문서 및 문서 로더
- 텍스트 분할기 (Text splitters)
- 임베딩 (Embeddings)
- 벡터 스토어 및 검색기 (Vector stores and retrievers)

### 문서 및 문서 로더 (Documents and Document Loaders)
LangChain은 Document 추상화를 제공합니다. 이 객체는 텍스트 단위와 관련 메타데이터를 나타내기 위해 설계되었습니다.  

Document의 주요 속성:  
- page_content: 문서의 내용을 나타내는 문자열입니다.  
- metadata: 문서의 출처, 다른 문서와의 관계 등 임의의 메타데이터를 포함하는 딕셔너리입니다.  
- id: (선택적) 문서를 식별하는 문자열입니다.  

metadata 속성은 문서의 출처, 문서 간 관계 및 기타 정보를 포함할 수 있습니다. 일반적으로 하나의 Document 객체는 더 큰 문서의 일부(청크)를 나타냅니다.

예제 문서 생성:
```python
from langchain_core.documents import Document

documents = [
    Document(
        page_content="개는 충성심과 친근함으로 잘 알려진 훌륭한 동반자입니다. ",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="고양이는 독립적인 반려동물로, 종종 자신만의 공간을 즐깁니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
]
```

그러나 LangChain 생태계는 문서 로더(document loaders)를 구현하여 수백 가지 일반적인 소스와 통합할 수 있습니다. 이를 통해 이러한 소스의 데이터를 AI 애플리케이션에 쉽게 통합할 수 있습니다.


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## **문서 로드하기**

PDF 파일을 `Document` 객체의 시퀀스로 로드해 보겠습니다. LangChain 저장소에 [예제 PDF](https://github.com/langchain-ai/langchain/tree/master/docs/docs/example_data)가 있습니다. 이 PDF는 **2023년 나이키(Nike)의 10-K 보고서**입니다.  

In [5]:
# pip install pypdf

In [6]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example_data/nke-10k-2023_korean.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

103


**`PyPDFLoader`** 는 **PDF의 각 페이지마다 하나의 `Document` 객체**를 로드합니다.  

각 객체에서 다음 정보를 쉽게 접근할 수 있습니다:  

- **페이지의 문자열 콘텐츠**  
- **파일 이름과 페이지 번호를 포함한 메타데이터**  

In [7]:
print(f"{docs[0].page_content[:200]}")
print()
print(docs[0].metadata)

목차 
미국 
증권거래위원회(SEC) 워싱턴 D.C. 20549 
양식 10-K 
(마크 1) 
☑ 2023 년 5 월 31 일에 종료되는 회계 연도에 대한 1934 년 증권거래법 섹션 13 또는 15(D)에 따른 연례 보고서 
또는 
☐ 1934 년 증권거래법(SECURITIES EXCHANGE ACT OF 1934)의 섹션 13 또는 15(D)에 따른 

{'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft® Word Microsoft 365용', 'creationdate': '2024-12-26T10:17:40+09:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2024-12-26T10:17:40+09:00', 'source': './example_data/nke-10k-2023_korean.pdf', 'total_pages': 103, 'page': 0, 'page_label': '1'}


## **텍스트 분할 (Splitting)**  

정보 검색 및 후속 질문-응답(question-answering) 작업을 하는데 **페이지 단위** 사용은 너무 거친 표현일 수 있습니다. 우리의 최종 목표는 **입력 쿼리에 답변할 수 있는 `Document` 객체를 검색하는 것**이므로, PDF를 더 세분화하면 문서의 해당되는 부분이 주변 텍스트에 의해 의미가 흐려지는 것을 방지할 수 있습니다.  

이를 위해 텍스트 분할기(text splitters)를 사용할 수 있습니다. 여기서는 **문자를 기준으로 분할하는 간단한 텍스트 분할기**를 사용할 것입니다.  

- 문서를 **1000자 단위로 분할**합니다.  
- 각 청크(chunk) 사이에는 **200자의 중첩(overlap)** 이 있습니다.  

**중첩**은 중요한 문맥(context)과 문장(statement)이 분리되는 것을 완화하는 데 도움이 됩니다.  

우리는 **RecursiveCharacterTextSplitter**를 사용할 것입니다. 이 분할기는 일반적인 구분자(예: 줄바꿈)를 사용하여 **재귀적으로 문서를 분할**하며, 각 청크가 적절한 크기가 될 때까지 반복합니다. **일반 텍스트 사용 사례에서 권장되는 텍스트 분할기입니다.**  

### **추가 설정**  
- `add_start_index=True`로 설정하면 **각 분할된 `Document`가 원본 `Document` 내에서 시작하는 문자 인덱스**가 **메타데이터 속성(`start_index`)** 으로 보존됩니다.  

In [8]:
# RecursiveCharacterTextSplitter를 가져옵니다.
# 이 도구는 텍스트를 재귀적으로 분할하며, 일반적인 구분자(예: 줄바꿈)를 사용합니다.
from langchain_text_splitters import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,              # 각 청크(chunk)의 최대 문자 수
    chunk_overlap=200,           # 청크 간 중첩(overlap) 문자 수
    add_start_index=True        # 각 청크의 시작 인덱스를 메타데이터로 추가
)

# 문서를 분할합니다. 'docs'는 분할할 원본 문서 목록입니다.
all_splits = text_splitter.split_documents(docs)

# 분할된 문서의 총 개수
len(all_splits)

283

## **임베딩 (Embeddings)**  

**벡터 검색(Vector Search)** 은 비정형 데이터(예: 비정형 텍스트)를 저장하고 검색하는 일반적인 방법입니다. 핵심 아이디어는 텍스트와 연결된 **숫자 벡터(Numeric Vectors)** 를 저장하는 것입니다.  

쿼리(Query)가 주어지면, 이를 **동일한 차원의 벡터로 임베딩(Embedding)** 하고 **벡터 유사도 측정(metric)** (예: **코사인 유사도**)을 사용하여 관련 텍스트를 식별할 수 있습니다.   

### **모델 선택하기**  

원하는 임베딩 모델을 선택해 사용합니다.  
예를 들어, OpenAI, Hugging Face, Cohere 등 다양한 제공자를 사용할 수 있습니다.  

**임베딩 모델을 선택한 후 벡터 스토어(Vector Store)에 텍스트를 저장하고 검색**할 수 있습니다.

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1158e6dd0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x1158e6e90>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [10]:
# 쿼리 텍스트를 벡터로 임베딩합니다.
# 각 청크의 내용을 벡터로 변환하여 비교합니다.

# 첫 번째 청크(all_splits[0])의 내용을 벡터로 임베딩합니다.
vector_1 = embeddings.embed_query(all_splits[0].page_content)

# 두 번째 청크(all_splits[1])의 내용을 벡터로 임베딩합니다.
vector_2 = embeddings.embed_query(all_splits[1].page_content)

# 두 벡터의 길이가 동일한지 확인합니다.
assert len(vector_1) == len(vector_2), "벡터 길이가 일치하지 않습니다!"

# 벡터의 길이를 출력합니다.
print(f"생성된 벡터의 길이: {len(vector_1)}\n")

# 첫 번째 벡터의 일부(처음 10개 요소)를 출력합니다.
print(vector_1[:10])

생성된 벡터의 길이: 1536

[0.02033882401883602, 0.02913365699350834, 0.023025788366794586, 0.02973075956106186, 0.01828628219664097, 0.007998693734407425, -0.016022266820073128, 0.003414683509618044, 0.043513890355825424, -0.027665777131915092]


텍스트 임베딩을 생성하는 모델을 준비했다면, 이제 이를 효율적인 **유사도 검색(Similarity Search)** 을 지원하는 **벡터 스토어(Vector Stores)** 에 저장할 수 있습니다.

---

## **벡터 스토어(Vector Stores)**  

LangChain의 [**VectorStore**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) 객체는 **텍스트 및 `Document` 객체를 저장**하고, 다양한 **유사도 메트릭(Similarity Metrics)** 을 사용해 쿼리를 수행할 수 있는 메서드를 포함합니다.  

이 객체들은 종종 **임베딩(Embedding)** 모델로 초기화되며, 해당 모델은 **텍스트 데이터를 숫자 벡터로 변환하는 방법**을 결정합니다.   

- 일부 벡터 스토어는 **클라우드 제공업체(Cloud Providers)** 에서 호스팅되며 특정 **자격 증명(Credentials)** 이 필요합니다.  
- 일부 벡터 스토어(예: **Postgres**)는 독립적인 인프라에서 실행되거나 로컬 또는 서드파티 플랫폼을 통해 운영될 수 있습니다.  
- 일부 벡터 스토어는 **인메모리(In-Memory)** 로 실행되어 가벼운 작업에 적합합니다.  

In [14]:
# !pip install -q chromadb
# !pip install -q langchain-chroma

In [15]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)
vector_store

벡터 저장소를 인스턴스화했으므로 이제 문서를 인덱싱할 수 있습니다.

In [16]:
# 문서 목록을 벡터 스토어에 추가하고, 각 문서에 대한 고유 식별자(ID) 반환
# - documents: 추가할 문서 목록 (여기서는 'all_splits'로 분할된 문서 목록)

ids = vector_store.add_documents(documents=all_splits)
ids[:5]

['ddfc7292-136d-4ded-b10c-24e836dfc2b7',
 '8dd50eb2-1215-4a19-a24c-cac2dd17e6cc',
 'f7319a89-2d79-4f51-96df-cf07fc0900e7',
 '4e31d26e-e85b-4fc9-8fca-628b2b86896d',
 'c836e7e1-5e61-4c53-aaef-a87e48267a41']

## **VectorStore 쿼리하기**  

**`VectorStore`** 가 문서를 포함하고 있는 상태에서, 우리는 이를 쿼리할 수 있습니다.  
[**VectorStore**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html)는 다음과 같은 쿼리 메서드를 제공합니다:  

- **동기(Synchronous) 및 비동기(Asynchronous) 방식**  
- **문자열(String) 쿼리 또는 벡터(Vector) 기반 쿼리**  
- **유사도 점수(Similarity Scores) 반환 여부 선택**  
- **유사도(Similarity)** 및 [**최대 주변부 중요도(Maximum Marginal Relevance)**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.max_marginal_relevance_search)를 사용하여 검색된 결과의 다양성을 균형 있게 조정  

일반적으로 이 메서드들은 출력값으로 [**Document**](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 객체 목록을 포함합니다.  

| **메서드**                     | **입력**           | **출력**                           | **사용 시점**                              |
|--------------------------------|------------------|---------------------------------|--------------------------------------|
| `similarity_search()`          | 텍스트(Query)    | 유사한 문서 리스트                 | 일반적인 유사도 검색                      |
| `similarity_search_with_score()` | 텍스트(Query)    | `(문서, 유사도 점수)` 리스트       | 유사도 점수를 포함하여 검색하고 싶을 때     |
| `similarity_search_by_vector()` | 사전 계산된 벡터 | 유사한 문서 리스트                 | 벡터를 직접 활용한 검색                   |


---

**임베딩(Embeddings)** 은 일반적으로 텍스트를 **'밀집 벡터(Dense Vector)'** 로 표현합니다. 이로 인해 **의미가 유사한 텍스트는 기하학적으로 가까운 위치**에 있게 됩니다.  
이를 통해 **문서에 사용된 특정 키워드를 알지 못해도** 질문을 입력하는 것만으로 관련 정보를 검색할 수 있습니다.  

In [17]:
results = vector_store.similarity_search(
    "Nike는 미국에 몇 개의 유통 센터를 가지고 있습니까?"
)

print(results[0])

page_content='애널리스트의 추정치는 종종 우리의 추정치 또는 기대와 다릅니다. 당사의 영업 실적이 공개 시장 분석가 및 투자자의 추정치 또는 기대치를 밑돌면 주가가 하락할 수 
있습니다. 과거에는 증권 시장 가격이 하락한 후 NIKE 및 기타 회사를 상대로 증권 집단 소송이 제기되었습니다. 어떤 이유로든 주가가 변동하는 경우 향후 이러한 유형의 
소송에 연루될 수 있습니다. 모든 소송은 평판 손상, 상당한 비용 및 비즈니스를 성공적으로 운영하는 데 필요한 경영진의 관심과 자원의 전환을 초래할 수 있습니다. 
2023 양식 10-K 23 
 
항목 1B. 해결되지 않은 직원 의견 
없음. 
항목 2. 속성 
다음은 NIKE 가 소유하거나 임대한 주요 자산에 대한 요약입니다. 
NIKE 가 소유하고 미국 오레곤 주 비버튼 근처에 위치한 NIKE World Campus는 약 400 에이커의 부지로 40 개 이상의 건물로 구성되어 있으며, 인접한 임대 부동산과 함께 
세계 본부 역할을 하며 거의 모든 부문에 서비스를 제공하는 관리, 연구, 디자인, 개발, 마케팅, 재무 및 기타 관리 기능에 종사하는 약 11,400명의 직원이 근무하고 있습니다. 
우리는 네덜란드 힐베르쉼(Hilversum)에 있는 비슷하지만 더 작은 관리 시설을 임대하여 유럽, 중동 및 아프리카 지역의 본부 역할을 하고 미국 외 지역의 특정 브랜드 기능을 
관리합니다. 
작업. 우리는 또한 중국 상하이에 있는 사무실 단지를 임대하고 있으며, 이 사무실은 중화권 지역의 본사로, 다른 기능 중에서도 도매, NIKE Direct 및 이 지역에서 머천다이징 
전략을 구현하는 데 중점을 둔 직원들이 사용하고 있습니다. 
미국에는 NIKE 가 8 개의 주요 유통 센터를 보유하고 있습니다. 5 개는 테네시 주 멤피스 또는 그 인근에 위치하고 있으며, 그 중 2 개는 소유이고 3 개는 임대입니다.' metadata={'producer': 'Microsoft® Word Microsoft 365용', 'st

<br>
- 유사도 점수 (Return scores) 반환

In [18]:
# 'similarity_search_with_score' 메서드는 쿼리와 가장 유사한 문서를 반환합니다.
# 반환값: (문서 객체, 유사도 점수) 쌍의 리스트
results = vector_store.similarity_search_with_score("2023년 Nike의 매출은 얼마였나요?")

# 첫 번째 결과를 추출합니다.
doc, score = results[0]

# 유사도 점수 출력
print(f"Score: {score}\n")

# 검색된 문서 내용 출력
print(doc)

Score: 0.6327033042907715

page_content='매장이 지난 1 년 동안 영구적으로 재배치되지 않았다는 세 가지 요건이 모두 충족된 NIKE 소유의 인라인 및 공장 매장의 매출로 구성됩니다. 비교 가능한 매장 매출에는 
COVID-19 로 인해 해당 기간 동안 일시적으로 폐쇄된 매장의 매출이 포함됩니다. 비교 가능한 매장 매출은 경영진과 투자자가 기존 NIKE 소유의 인라인 및 공장 매장의 
실적을 이해하는 데 유용한 정보라고 생각하는 성과 지표를 나타냅니다. 경영진은 재무 및 운영 결정을 내릴 때 이 지표를 고려합니다. 비교 가능한 매장 매출을 계산하는 
방법은 소매 업종에 따라 다릅니다. 따라서 이 지표에 대한 당사의 계산은 다른 회사에서 사용하는 유사한 제목의 지표와 비교할 수 없을 수 있습니다. 
2023 양식 10-K 30 
 
운영 결과 
(백만 달러 단위, 주당 데이터 제외) 2023 회계연도 2022 회계연도 변동률(%) 2021 회계연도 변동률(%) 
수익 $ 51,217  $ 46,710  10 % $ 44,538  5 % 
매출 원가  28,925   25,231  15 % 24,576  3 % 
매출 총이익  22,292   21,479  4 % 19,962  8 % 
총수익  43.5 %  46.0 %  44.8 %  
수요 창출 비용  4,060   3,850  5 % 3,114  24 % 
영업간접비  12,317   10,954  12 % 9,911  11 % 
총 판매 및 관리 비용  16,377   14,804  11 % 13,025  14 % 
매출 대비 %  32.0 %  31.7 %  29.2 %' metadata={'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2024-12-26T10:17:40+09:00', 'source': './example_data/nke-10k-2023_korean.pdf', 'subject': 'Form 10-K fi

----------
내장된 쿼리와의 유사성을 기준으로 문서 반환:

In [19]:
embedding = embeddings.embed_query("2023년 Nike의 마진은 어떻게 영향을 받았나요?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='포인트 감소했습니다. 매출은 주로 남성, 여성 및 Jordan 브랜드의 성장으로 인해 증가했습니다. NIKE Direct 의 매출은 23%의 디지털 매출 성장과 28%의 유사 매장 매출 
성장에 힘입어 22% 증가했습니다. 
• 신발 매출은 통화 중립 기준으로 19% 증가했는데, 이는 주로 남성, 여성 및 요르단 브랜드의 매출 증가에 기인합니다. 신발 판매량은 16% 증가했으며, 신발 한 켤레당 
ASP 가 높아지면서 신발 매출 성장에 약 3% 포인트 기여했습니다. 쌍당 ASP 가 높은 것은 주로 정가 ASP 가 높아지고 NIKE Direct 가 성장했기 때문이며, NIKE Direct 
ASP 가 낮아지면서 부분적으로 상쇄되었습니다. 
• 의류 매출은 통화 중립 기준으로 13% 증가했는데, 이는 주로 남성복 매출 증가에 기인합니다. 의류 판매량은 9% 증가했으며, 단위당 ASP 가 높아지면서 의류 매출 
성장률이 약 4% 포인트 증가했습니다. 단위당 ASP 가 높은 것은 주로 정가 및 오프 가격 ASP 가 높았기 때문이며, NIKE Direct ASP가 낮아지면서 부분적으로 
상쇄되었습니다. 
보고된 EBIT 는 매출 증가와 다음과 같은 이유로 2% 증가했습니다. 
• 약 190bp 의 매출총이익 감소는 주로 제품 비용 상승으로 인한 것으로, 제품 믹스 및 투입 비용 증가, 표준 외화 환율의 불리한 변화를 반영합니다. 이는 제품 믹스와 
전략적 가격 책정 조치로 인해 더 높은 정가 ASP , 순 할인으로 부분적으로 상쇄되었습니다. 
• 판매 및 관리비는 운영 간접비 증가와 수요 창출 비용 증가로 인해 8% 증가했습니다. 영업간접비는 주로 임금 관련 비용 증가와 NIKE Direct 변동 비용 증가로 인해 
증가했으며, 이는 환율의 우호적인 변동으로 일부 상쇄되었습니다. 수요 창출 비용은 주로 스포츠 마케팅 비용 증가와 광고 및 마케팅 비용 증가로 인해 증가했으며, 이는 
환율의 우호적인 변화로 일부 상쇄되었습니다. 
2023 양식 1

## **검색기 (Retrievers)**  

LangChain의 **`VectorStore` 객체**는 [**Runnable**](https://python.langchain.com/api_reference/core/index.html#langchain-core-runnables)의 서브클래스가 아닙니다. 반면, **[Retrievers](https://python.langchain.com/api_reference/core/index.html#langchain-core-retrievers)** 는 **Runnable**의 서브클래스입니다. 따라서 **동기(Synchronous)** 및 **비동기(Asynchronous)** `invoke`와 `batch` 작업과 같은 표준 메서드 집합을 구현합니다.  

### **검색기 vs 벡터 스토어**  
- **벡터 스토어(VectorStore)**: 벡터화된 데이터를 저장하고 쿼리합니다.  
- **검색기(Retriever)**: 벡터 스토어를 활용하거나 외부 API와 같은 **비벡터 데이터 소스**와도 통신할 수 있습니다.  

**벡터 스토어(Vectorstores)** 는 **`as_retriever`** 메서드를 구현하며, 이를 통해 **[VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html)** 를 생성할 수 있습니다.  

이 검색기(Retriever)는 특정 **`search_type`** 및 **`search_kwargs`** 속성을 포함하며, 이를 통해 **벡터 스토어의 어떤 메서드를 호출할지**, 그리고 **어떻게 매개변수를 설정할지**를 정의합니다.  

| 항목                 | `similarity`                     | `mmr` (Maximum Marginal Relevance)      | `similarity_score_threshold`              |
| ------------------ | -------------------------------- | --------------------------------------- | ----------------------------------------- |
| 🔍 검색 기준           | 질의(query)와 가장 **유사한 벡터**를 단순히 선택 | 유사도 + **결과 간 다양성** 고려                   | 유사도 기준을 **임계값 이상**으로 제한                   |
| 🎯 목적              | 가장 관련성 높은 문서 찾기                  | 중복 없이 **정보 다양성 확보**                     | 너무 유사하지 않은 문서 제거                          |
| ⚙️ 주요 파라미터         | `k`: 반환할 문서 수                    | `k`: 개수<br>`lambda_mult`: 다양성 비율 (0\~1) | `score_threshold`: 최소 유사도<br>`k`: 최대 문서 수 |
| 📈 적합한 경우          | 단순 정보 검색                         | 요약, RAG 등에서 다양한 관점 필요할 때                | 정확도 높은 문서만 사용하고 싶을 때                      |
| 🚫 주의점             | 중복된 내용이 나올 수 있음                  | 다양성 높이려다 **정확도 저하** 가능                  | `score_threshold` 너무 높으면 결과 없을 수도 있음      |


예를 들어, 위의 예제를 다음과 같이 재현할 수 있습니다:  

In [20]:
# `as_retriever` 메서드를 사용하여 벡터 스토어를 검색기 객체로 변환합니다.
retriever = vector_store.as_retriever(
    search_type="similarity",  # 검색 유형을 'similarity'(유사도)로 설정
    search_kwargs={"k": 1},  # 상위 1개의 유사한 결과만 반환하도록 설정
)

# MMR 방식 (Maximum Marginal Relevance) - 검색 결과 다양성을 증가
retriever_mmr = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1, "lambda_mult": 0.5}  # lambda_mult 조절 가능 (0~1 사이 값)
)

# Similarity Score Threshold 방식 - 일정 유사도 이상인 문서만 반환
retriever_threshold = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8, "k": 1}  # 0.8 이상의 유사도 점수를 가진 문서만 반환
)

# 여러 개의 쿼리를 동시에 실행하여 검색 결과를 반환합니다.
retriever.batch(
    [
        "2023년 Nike의 매출은 얼마였나요?",  # 첫 번째 쿼리: 2023년 Nike의 매출 정보
        "2023년 Nike의 마진은 어떻게 영향을 받았나요?",  # 두 번째 쿼리: 2023년 Nike의 마진 영향
    ],
)

[[Document(id='99eefe1f-a5e1-475f-989a-b8df29c95df8', metadata={'creator': 'Microsoft® Word Microsoft 365용', 'moddate': '2024-12-26T10:17:40+09:00', 'total_pages': 103, 'page_label': '27', 'source': './example_data/nke-10k-2023_korean.pdf', 'producer': 'Microsoft® Word Microsoft 365용', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'start_index': 890, 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'title': '0000320187-23-000039', 'keywords': '0000320187-23-000039; ; 10-K', 'creationdate': '2024-12-26T10:17:40+09:00', 'page': 26}, page_content='매장이 지난 1 년 동안 영구적으로 재배치되지 않았다는 세 가지 요건이 모두 충족된 NIKE 소유의 인라인 및 공장 매장의 매출로 구성됩니다. 비교 가능한 매장 매출에는 \nCOVID-19 로 인해 해당 기간 동안 일시적으로 폐쇄된 매장의 매출이 포함됩니다. 비교 가능한 매장 매출은 경영진과 투자자가 기존 NIKE 소유의 인라인 및 공장 매장의 \n실적을 이해하는 데 유용한 정보라고 생각하는 성과 지표를 나타냅니다. 경영진은 재무 및 운영 결정을 내릴 때 이 지표를 고려합니다. 비교 가능한 매장 매출을 계산하는 \n방법은 소매 업종에 따라 다릅니다. 따라서 이 지표에 대한 당사의 계산은 다른 회사에서 사용하는 유사한 제목의 지표와 비교할 수 없을 수 있습니다. \n2023 양식 10-K 30 \n \